<a href="https://colab.research.google.com/github/dannyyoon0303/ECG-Project/blob/main/Modified_Model_(1D_CNN_%2B_Physical_Quantity).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install neurokit2
!pip install tensorflow==2.8
!apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 5.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 668.3 MB 17 kB/s 
     |████████████████████████████████| 462 kB 5.3 MB/s 
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.8.2+zzzcolab20220719082949
    Uninstalling tensorflow-2.8.2+zzzcolab20220719082949:
      Successfully uninstalled tensorflow-2.8.2+zzzcolab20220719082949
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following packages will be REMOVED:
  libcudnn8-dev
The following held packages will be changed:
  libcudnn8
The following packages will be upgraded:
  libcud

In [ ]:
import pandas as pd
import numpy as np
import neurokit2 as nk
import tensorflow as tf 
from keras import optimizers, losses, activations, models
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from keras.layers import Dense, Input, Dropout, Convolution1D, MaxPool1D, GlobalMaxPool1D, GlobalAveragePooling1D, \
    concatenate, Add
from sklearn.metrics import f1_score, accuracy_score

In [ ]:
train_file_path = "/content/drive/MyDrive/[Project] 윤정현 - ECG/Data/full_training_set(size=87554).csv"
test_file_path = "/content/drive/MyDrive/[Project] 윤정현 - ECG/Data/full_test_set(size=21892).csv"

df_train = pd.read_csv(train_file_path, header=None)[1:].astype(float)
print(df_train)
df_train = df_train.sample(frac=1)
df_test = pd.read_csv(test_file_path, header=None)[1:].astype(float)

Y = np.array(df_train[187].values).astype(np.int8)
X_ecg = np.array(df_train[list(range(187))].values)[..., np.newaxis]
X_physical = np.array(df_train[list(range(188, 193))].values)[..., np.newaxis]

Y_test = np.array(df_test[187].values).astype(np.int8)
X_ecg_test = np.array(df_test[list(range(187))].values)[..., np.newaxis]
X_physical_test = np.array(df_test[list(range(188, 193))].values)[..., np.newaxis]

            0         1         2         3         4         5         6    \
1      1.000000  0.693965  0.336207  0.271552  0.250000  0.211207  0.250000   
2      0.139535  0.682171  0.782946  0.000000  0.170543  0.472868  0.604651   
3      0.944751  0.906077  0.646409  0.414365  0.209945  0.093923  0.060773   
4      0.956221  0.898618  0.826037  0.740783  0.625576  0.531106  0.414747   
5      0.956416  0.869249  0.290557  0.000000  0.152542  0.251816  0.285714   
...         ...       ...       ...       ...       ...       ...       ...   
87550  0.968750  1.000000  0.718750  0.368056  0.059028  0.059028  0.142361   
87551  1.000000  0.676157  0.064057  0.021352  0.110320  0.092527  0.064057   
87552  0.952381  0.935374  0.632653  0.241497  0.010204  0.000000  0.112245   
87553  0.835206  0.838951  0.632959  0.284644  0.011236  0.041199  0.153558   
87554  1.000000  0.887978  0.494536  0.245902  0.147541  0.125683  0.084699   

            7         8         9    ...  183  184 

In [ ]:
def get_model():
    nclass = 5
    inp_ecg = Input(shape=(187, 1))
    img_1 = Convolution1D(16, kernel_size=5, activation=activations.relu, padding="valid")(inp_ecg)
    img_1 = Convolution1D(16, kernel_size=5, activation=activations.relu, padding="valid")(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = Dropout(rate=0.1)(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = Dropout(rate=0.1)(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = Convolution1D(32, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = MaxPool1D(pool_size=2)(img_1)
    img_1 = Dropout(rate=0.1)(img_1)
    img_1 = Convolution1D(256, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = Convolution1D(256, kernel_size=3, activation=activations.relu, padding="valid")(img_1)
    img_1 = GlobalMaxPool1D()(img_1)
    img_1 = Dropout(rate=0.2)(img_1)
    
   
    dense_1 = Dense(64, activation=activations.relu, name="dense_1")(img_1)
    inp_physical = Input(shape=(5))
    concat_1 = concatenate([dense_1, inp_physical])
    dense_2 = Dense(64, activation=activations.relu, name="dense_2")(concat_1)
    dense_3 = Dense(nclass, activation=activations.softmax, name="dense_3_mitbih")(dense_2)

    model = models.Model(inputs=[inp_ecg, inp_physical], outputs=dense_3)
    opt = tf.optimizers.Adam(0.001)

    model.compile(optimizer=opt, loss=losses.sparse_categorical_crossentropy, metrics=['acc'])
    model.summary()
    return model


In [ ]:
model = get_model()
file_path = "modified_cnn_mitbih.h5"
checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="val_acc", mode="max", patience=5, verbose=1)
redonplat = ReduceLROnPlateau(monitor="val_acc", mode="max", patience=3, verbose=2)
callbacks_list = [checkpoint, early, redonplat]  # early

history = model.fit([X_ecg, X_physical], Y, epochs=1000, verbose=2, callbacks=callbacks_list, validation_split=0.1)
model.load_weights(file_path)


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 187, 1)]     0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 183, 16)      96          ['input_1[0][0]']                
                                                                                                  
 conv1d_1 (Conv1D)              (None, 179, 16)      1296        ['conv1d[0][0]']                 
                                                                                                  
 max_pooling1d (MaxPooling1D)   (None, 89, 16)       0           ['conv1d_1[0][0]']               
                                                                                              

In [ ]:
pred_test = model.predict([X_ecg_test, X_physical_test])
pred_test = np.argmax(pred_test, axis=-1)

f1 = f1_score(Y_test, pred_test, average="macro")

print("Test f1 score : %s "% f1)

acc = accuracy_score(Y_test, pred_test)

print("Test accuracy score : %s "% acc)

Test f1 score : 0.9112856896270202 
Test accuracy score : 0.9836926731226019 


In [ ]:
from sklearn.metrics import precision_recall_fscore_support as score

precision, recall, fscore, support = score(Y_test, pred_test)
print("Precision : %s, %s" % (np.mean(precision), precision))
print("Recall : %s, %s" % (np.mean(recall), recall))
print("F1 Score : %s, %s" % (np.mean(fscore), fscore))

Precision : 0.9409927172866126, [0.98805741 0.90677966 0.95655221 0.86231884 0.99125547]
Recall : 0.8858892089111169, [0.99547411 0.76978417 0.94267956 0.7345679  0.9869403 ]
F1 Score : 0.9112856896270202, [0.9917519  0.83268482 0.94956522 0.79333333 0.98909318]


In [ ]:
df = pd.DataFrame(history.history)
df.to_csv('/content/drive/MyDrive/[Project] 윤정현 - ECG/Result/model1.csv')